# Input ZIP Code Data for Missing Values
2023-10-04<br>
Evangeline Chang

In [1]:
import pandas as pd
import os

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
crime = pd.read_csv('data/crime_data_2020_zip_na.csv')

In [3]:
crime = crime.sort_values(by=['LOCATION', 'Cross Street'], ignore_index=True)
print(crime['ZIP'].isna().sum())

5


Automate the process of inputting zip code data for rows with missing latitude and longitude data

In [4]:
def fill_same(in_file):
    rows_to_update = []

    for index, row in in_file.iterrows():
        if pd.isna(row['ZIP']):
            prev_index = index - 1
            next_index = index + 1

            if prev_index >= 0 and in_file.at[index, 'LOCATION'] == in_file.at[prev_index, 'LOCATION']:
                if pd.isna(in_file.at[index, 'Cross Street']) and pd.isna(in_file.at[prev_index, 'Cross Street']):
                    rows_to_update.append((index, in_file.at[prev_index, 'ZIP']))
                elif in_file.at[index, 'Cross Street'] == in_file.at[prev_index, 'Cross Street']:
                    rows_to_update.append((index, in_file.at[prev_index, 'ZIP']))

            elif next_index < len(in_file) and in_file.at[index, 'LOCATION'] == in_file.at[next_index, 'LOCATION']:
                if pd.isna(in_file.at[index, 'Cross Street']) and pd.isna(in_file.at[next_index, 'Cross Street']):
                    rows_to_update.append((index, in_file.at[next_index, 'ZIP'])) 
                elif in_file.at[index, 'Cross Street'] == in_file.at[next_index, 'Cross Street']:
                    rows_to_update.append((index, in_file.at[next_index, 'ZIP']))

    for index, zip_code in rows_to_update:
        in_file.at[index, 'ZIP'] = zip_code

In [5]:
num_iterations = 2   #run fill_same(crime) several times 

for _ in range(num_iterations):
    fill_same(crime)

In [6]:
crime['ZIP'].isna().sum()  # the rest has to be input manually

0

In [7]:
output_directory = 'data/'

if not os.path.exists(output_directory):
    os.makedirs(output_directory)
    
crime.to_csv('data/crime_data_2020_zipcode_final_2.csv')